In [ ]:
## importing libraries
# import pandas as pd
import numpy as np
import os
import openai
import spacy
from tqdm import tqdm
#gpt3.5
# azure_base = "https://gptopenaidev.openai.azure.com/"
# azure_key = "6a0167093c874df6bcac4d3f9c414cd5"
# openai_api_version_t = "2023-03-15-preview"

# #gpt 4
# openai.api_type = "azure"
# azure_base = "https://genaicoedevopenaieast01.openai.azure.com/openai"
# openai_api_version_t = "2024-08-01-preview"
# azure_key = "351e93b1489a45769b3dadac8a0f78b9"


#gpt 4
openai.api_type = "azure"
azure_base = "https://genaicoedevopenaieast01.openai.azure.com/openai"
openai_api_version_t = "2023-03-15-preview"
azure_key = "a20ac46cd86c4c49bb795b713e814afb"

openai.api_type = "azure"
openai.api_version = openai_api_version_t 
openai.api_base = azure_base # Your Azure OpenAI resource's endpoint value.
# openai.
openai.api_key = azure_key


API_KEY = "a20ac46cd86c4c49bb795b713e814afb"
# RESOURCE_NAME =  "azure"
DEPLOYMENT_NAME = "gpt-4" 
API_VERSION = "2023-03-15-preview"  # Example, check Azure docs for the latest version
ENDPOINT =  "https://gpt4openaidev.openai.azure.com/"
API_BASE = "https://gpt4openaidev.openai.azure.com/"



In [ ]:
pip install openai==0.28

In [3]:
import pandas as pd

In [ ]:
pip install --upgrade numpy pandas

In [18]:
## Loading files
df = pd.read_csv("final_df_16Nov.csv")
nlp = spacy.load("en_core_web_sm")

In [ ]:
pip show spacy

In [ ]:
pip install spacy --upgrade

In [ ]:
pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [19]:
## API Function
def api_call(input_str, temp, prompt):
    temp = temp
    prompt = prompt
    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo-16k",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": "Hello, how are you?"}
            # {"role": "user", "content": input_str}
        ], 
        temperature = temp
    )
    return (response)

In [8]:
def input_generator(que,ans):
    input_str = "Question = " + que + "\n ------------------------------------------------------------"
    input_str = input_str + "\n" +"Answer = " + " = " + ans + "\n" + "----------------------------------------------------"
    return input_str

In [9]:
def keyword_generator(ans):
    keywords_string = ""
    doc = nlp(ans)
    for i in doc.ents:
        keywords_string += " " + str(i) + ","
    return keywords_string

In [10]:
def prompt_generator(keywords_string):
    prompt = """
    Question and Answer are provided, Question starts with 'Question' token, Answer starts with 'Answer' token. 
    Extract the below points:
    1. Capture all the suggestions and comments provided
    2. Also the suggestions/comments given related to these keywords '{0}'
    3. Consider each and every point important and try to cover most of the information as you can.
    """.format(keywords_string)
    return(prompt)

In [11]:
def grouping_responses(question, response_list):
    grouped_responses = "Question = " + question + "\n ----------------------------------------------------------------"
    for i in response_list:
        grouped_responses += "\n " + i
        grouped_responses += "\n ---------------------------------------------------------------------------------"
    return grouped_responses

In [12]:
df.columns[16]

'The PSS team is always looking at ways to improve. Do you have other comments or suggestions on how we continue to improve LIBTAYO Surround or the PJN / PLR?'

In [13]:
question_list = [df.columns[15]]

In [14]:
question_list

['Do you feel you have appropriate reference or training resources if you have questions?']

In [15]:
temper = 0.5

In [20]:
outputs_2 = ''
for question in tqdm(question_list):
    outputs = []
    answers_list = df[question].values
    for answer in answers_list:
        if str(answer) != 'nan':
            input_str = input_generator(question,answer)
            keywords = keyword_generator(answer)
            prompt = prompt_generator(keywords)
            output = api_call(input_str, temper, prompt)
            outputs.append(output['choices'][0]['message']['content'])
            
    consolidated_response = grouping_responses(question, outputs)
    
    print(consolidated_response)
    
    
    # prompt_2 = """Question and Answers are provided. Question starts with 'Question' token, Answer starts after '-' line. 
    # 1. Provide a indepth analysis by combining all the points mentioned in the answers. 
    # 2. Consider each and every point important and try to cover most of the information
    # as you can."""
    # temper_2 = 0.1
    # summary = api_call(consolidated_response, temper_2, prompt_2)
    
    # outputs_2 = summary['choices'][0]['message']['content']
    # # outputs_2 += "\n"+ "Question = " + question + "\n ----------------------------------------------------------------"
    # # outputs_2 += "\n " + (summary['choices'][0]['message']['content']) + "\n ---------------------------------------"



  0%|          | 0/1 [00:00<?, ?it/s]


InvalidRequestError: Resource not found

In [102]:
print(outputs_2)


Question = Do you feel you have appropriate reference or training resources if you have questions?
 ----------------------------------------------------------------
 The respondents generally feel that they have appropriate reference or training resources for their questions. They have mentioned various resources such as an internal ORM resource, which is like a new start guide or a refresher guide, rolled out about a month and a half ago. They also mentioned an internal website managed by Carolyn Crane, which is like a market access for all trainings and things. However, they have not personally used this resource. There was also a mention of a training website, the name and acronym of which have been changed recently, but the respondent couldn't recall it.

In terms of personal support, the respondents appreciate the availability of certain individuals for answering questions. They appreciate Sid's availability for answering questions and Mansour's graciousness with his time to prov

In [103]:
# 1. Seperately analyse the answers of the questions '{0}'
# 3. Provide the response in bullets
# 3. Provide the response in bullets, in a professional way

In [104]:
prompt_2 = """Question and Answers are provided. Question starts with 'Question' token, Answer starts after '-' line. 
1. Provide a precise analysis by combining all the points mentioned in the answers neglecting the ones with similarity. 
2. Capture all the information, each and every point is important
3. Provide the response in a bullet points
"""

temper_2 = 0.5
summary = api_call(outputs_2, temper_2, prompt_2)

In [105]:
print(summary['choices'][0]['message']['content'])

- Respondents generally feel they have appropriate reference or training resources for their queries.
- Resources mentioned include an internal ORM resource, a new start guide or refresher guide rolled out recently.
- An internal website managed by Carolyn Crane serves as a market access for all trainings, although not personally used by respondents.
- A training website, recently renamed, was also mentioned but its specific name was not recalled.
- Personal support is highly appreciated, with individuals like Sid and Mansour available for answering questions.
- A weekly forum with a PSS team member is seen as a valuable resource for addressing queries, with Mansour and Jen specifically noted as helpful.
- Respondents tend to keep all emails, tips, and other communication for reference.
- The ServiceNow request presentation was found to be helpful.
- Respondents don't see any obstacles in acquiring necessary knowledge or finding answers.
- There is some uncertainty about the location a

In [106]:
prompt_2 = """Question and Answers are provided. Question starts with 'Question' token, Answer starts after '-' line. 
1. Provide a precise summary by combining all the points mentioned in the answers neglecting the ones with similarity. 
2. Capture all the information,'' each and every point is important
3. Provide the response in a bullet points
"""

temper_2 = 0.5
summary = api_call(outputs_2, temper_2, prompt_2)

In [107]:
print(summary['choices'][0]['message']['content'])

- Respondents generally feel they have appropriate reference or training resources for their questions, including an internal ORM resource and an internal website managed by Carolyn Crane.
- They have not personally used the internal website, which serves as a market access for all trainings.
- A training website was mentioned, but its name and acronym have recently changed and were not recalled.
- Personal support is valued, with individuals like Sid and Mansour appreciated for their availability and quick responses.
- There is a weekly forum with a PSS team member always present to answer queries, with Mansour and Jen specifically mentioned as helpful.
- Respondents keep all emails, tips, and other information sent out for reference, and found the ServiceNow request presentation helpful.
- Despite finding the current resources and support system extremely helpful, some respondents are unsure about the location and nature of the resources.
- There were no specific comments or suggesti

In [49]:
# prompt_2 = """Question and Answers are provided. Question starts with 'Question' token, Answer starts after '-' line. 
# 1. Provide a precise analysis by combining all the points mentioned in the answers neglecting the ones with similarity. 
# 2. Capture all the information, each and every point is important
# 3. Provide the response in bullets
# """.format(question_list[0])

# temper_2 = 0.5
# summary = api_call(outputs_2, temper_2, prompt_2)

In [50]:
# print(summary['choices'][0]['message']['content'])